In [1]:
from datetime import datetime
import sys

In [2]:
import sqlite3
from sqlite3 import Error


def database_connection():
    try:
        con=sqlite3.connect('librarymanagement.db')
        return con
    except Error:
        print(Error)
        

con =database_connection()
cursorObj = con.cursor()

In [32]:
import sqlite3
from sqlite3 import Error
import datetime
from datetime import datetime
import sys


class library:
    
    def __init__(self,UserName,Password):
        print("welcome to Library ")
        self.UserName=UserName
        self.Password=Password
           
    def validatinguser(self):
        cursorObj.execute("SELECT * FROM users WHERE name=? and password=?",
        (UserName, Password))
        found = cursorObj.fetchone()
        if found:
            print(f"Welcome {UserName}")
            library.borrowReturnoraddmorebook()
        else:
            print("User does not exist or password does not match")
                
    def borrowReturnoraddmorebook(self):
        BorroworReturn=input("1.Do you want to borrow the books or 2.return the book or 3.addmorebooks")
        if(BorroworReturn=='1'): 
              
            data=library.query_books_borrowed_deatils(UserName)
            print(f"Books borrowd by {UserName} {data}")
            if len(data) >= 3:
                print("you cannot borrow more books , 3 is the max limit")
            else:
                rows= library.shelf()
                if len(rows) > 0:
                
                    print("======= ==================\n\n You can take a look at the books availabe in our library \n\n\n ==============")
                    df = pd.DataFrame.from_records(rows, columns =['BOOK ID','NAME OF THE BOOK', 'AUTHOR', 'PuUBLICATION'])
                    print(df)
                    count=1
                    while count < 4:
                        rows= library.shelf()
                        
                        if len(rows) > 0 :
                            choice=int(input("Enter the book id you wanna borrow:"))
                            library.updateShelf(choice)
                            library.books_borrowed_details(choice,UserName) 
                            print(f"yayyy!! you successfull borrowed bookid {choice}")
                            data1=library.query_books_borrowed_deatils(UserName)
                            print(data1)
                            if len(data1) >= 3: 
                                print("you cannot borrow more books , 3 is the max limit")
                                break
                            else:
                                user_choice=input("Do you want to borrow more book Yes/No")
                                if user_choice=='Yes' and len(data) < 4 :
                                    count = count+1
                                else:
                                    print("Thank you for using this service")
                                    break
                        else:
                            print("Sorry no more books availabe")
                            break
                else:
                    print("Sorry no books available")
        
        elif (BorroworReturn=='2'):
            data=library.query_books_borrowed_deatils(UserName)                    
            print(f"Books borrowd by {UserName} {data}")
            booksborrowed=True
            if len(data) > 0:
                while booksborrowed:                    
                    choice=int(input("Enter the book id you wanna return:"))
                    library.updateShelfonReturn(choice)
                    dateofbooktaken = library.checkBorrowedDate(choice)
                    for i in dateofbooktaken:
                        borrowedDate = (i[0])
                        todaysDate=library.getDATE()               
                    for i in todaysDate:
                        RetunedDate = (i[0])
                    
                    d1 = datetime.strptime(borrowedDate, "%Y-%m-%d")
                    
                    d2 = datetime.strptime(RetunedDate, "%Y-%m-%d")
                    numberofdays = (abs((d2 - d1).days))
                    if numberofdays > 20:
                        print(f"Sorry!!!!You need to pay fine of Rs 10 for keeping the book for {numberofdays} days")
                    library.books_returned(choice)
                    print("Thanks for Returning this book")
                    data=library.query_books_borrowed_deatils(UserName)  
                    print(f"You still have books to return {data}")
                    if len(data) > 0 :
                        ReturnMoreBooks=input("do you want to return more books Yes/No")
                        if(ReturnMoreBooks=="Yes"):
                            booksborrowed = True
                        else:
                            booksborrowed = False
                            print("Thanks!!Have a good day")
                            break
                    else:                            
                        print(f"you have nothing to return,thanks")
                        break
                            
                else:
                    print(f"you have nothing to return,thanks=====================")
        
        elif(BorroworReturn=='3'):
            count = 1
            while count < 5:
                book_name=input("please enter the book name:")
                author=input("please enter the author")
                publication=input("please enter the publication")
                library.add_more_books(book_name,author,publication)
                print("Hey!! you have successfully added the books")
                morebooks=input("Do you have more books to add Yes or No")
                if(morebooks=="Yes"):
                    count = count+1
                else:
                    print("Thank you!!!!!!!!!!!!!!!!")
                    break
        else:
            print("NOT a valid choice!")
            
   


    
    def Registeruser(self):
        try:
            cursorObj.execute("INSERT INTO users(name,password,contact) VALUES(?,?,?)", (UserName,Password,contact))
            con.commit()
            print("You are successfully registered to our library , you can relogin using your username and password")
        except Error:
            print(f"{Error},{UserName} already taken")    
            
    def shelf(self):
        cursorObj.execute("select book_id, name,Author,Publication from books where isavailable = 'Y'")
        rows = cursorObj.fetchall() 
        return rows
    
    def updateShelf(self,choice):
        cursorObj.execute("update books set isavailable='N' where book_id={}".format(choice))
        con.commit()
        
    def getDATE(self):
        cursorObj.execute("Select Date();")
        rows = cursorObj.fetchall() 
        return rows
        
    
    def books_borrowed_details(self,choice,UserName):
        data=library.getDATE()               
        for i in data:
            date = (i[0])
            try:
                cursorObj.execute("Insert into books_borrowed(book_id,User_name,Borroweddate) VALUES(?,?,?)", 
                                  (choice,UserName,date))
                con.commit()
            except:
                e = sys.exc_info()[0]
                print(e) 
                print("----your have entered a wrong choice-----")
            
    
     
    
    def query_books_borrowed_deatils(self,Username):
        #this is for selecting data from database
        cursorObj.execute("select * from books_borrowed where  User_name={}".format( "'" + Username + "'"))
        # this will only execute the sql syntax w.r.t cursor object
        rows = cursorObj.fetchall() # in order to visualize the date, use the fetchall() w.r.t cursor object and store in a variable -- this will return list of tuples 
        return rows #type is list 

    def updateShelfonReturn(self,choice):
        cursorObj.execute("update books set isavailable='Y' where book_id={}".format(choice))
        con.commit()
    
    def checkBorrowedDate(self,choice):
        cursorObj.execute("select Borroweddate from books_borrowed where  book_id={}".format(choice))
        # this will only execute the sql syntax w.r.t cursor object
        rows = cursorObj.fetchall() # in order to visualize the date, use the fetchall() w.r.t cursor object and store in a variable -- this will return list of tuples 
        return rows

    def books_returned(self,choice):     
        cursorObj.execute("delete from books_borrowed where book_id={}".format(choice))
        con.commit()
    
    def add_more_books(self,book_name,author,publication):
        try:
            cursorObj.execute("INSERT INTO books(name,Author,Publication,isavailable) VALUES(?,?,?,?)", (book_name,author,publication,"Y"))
            con.commit()
        except:
            e = sys.exc_info()[0]
            print(e)
    

    
    
   
        
    
                
    
if __name__ == "__main__":
    import pandas as pd
        
    user_input = input("1. Authenticate 2.Register")
    print(user_input)
    if user_input.lower() == '1':
        print("please enter your user name and password to authenticate")
        UserName=input("please enter your username: \t\t ")
        Password=input("please enter the passowrd: \t\t ")
        library = library(UserName,Password)
        library.validatinguser()
        
    elif user_input.lower() == '2':
        print("Alright we need below to register you into our library")
        UserName=input("please enter your username: \t\t ")
        Password=input("please enter the passowrd: \t\t ")
        contact=input("please enter the contactdetails: \t\t ")
        library = library(UserName,Password)
        library.Registeruser()
    else:
        print("Sorry you have entered wrong choice!!!")
        
        
            
            

1. Authenticate 2.Register1
1
please enter your user name and password to authenticate
please enter your username: 		 Manjunath
please enter the passowrd: 		 Manjunath1
welcome to Library 
Welcome Manjunath
1.Do you want to borrow the books or 2.return the book or 3.addmorebooks1
Books borrowd by Manjunath [(6, 'Manjunath', '2021-07-25')]
======= ==================

 You can take a look at the books availabe in our library 


   BOOK ID NAME OF THE BOOK    AUTHOR PuUBLICATION
0        3         Tampines  Tampines         AMAM
1        5       SonuMylove      Sonu         Sonu
2        7           Iammad       Anu          Ani
3        8            ohwow       wow          wow
4        9      mynameisAnu       Anu          Anu
5       10             ohno      ohno          aaa
6       13            Manju     Manju        Manju
Enter the book id you wanna borrow:3
yayyy!! you successfull borrowed bookid 3
[(6, 'Manjunath', '2021-07-25'), (3, 'Manjunath', '2021-07-25')]
Do you want to bor